In [4]:
import csv
from elasticsearch import Elasticsearch

In [5]:
es = Elasticsearch(
    ['http://localhost:9200'],
    http_auth=('elastic', '4862')
)

/tmp/ipykernel_7254/2209251241.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [3]:
import csv
from elasticsearch import Elasticsearch

index_name = "test_norelationnel"

# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "review": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                }
            },
            "date_review": {
                "type": "date",
                "format": "dd/MM/yyyy"
            },
            "cdiscount_response_date": {
                "type": "date",
                "format": "dd/MM/yyyy"
            },
            "cdiscount_response_content": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                }
            },
            "note": {
                "type": "integer"
            },
            "note_p": {
                "type": "float"
            },
            "polarite": {
                "type": "float"
            },
            "subjectivite": {
                "type": "float"
            }
        }
    }
}



es.indices.create(index=index_name, body=mapping, ignore=400)

with open('/home/jben/Documents/CDE_Satisfaction2023/data/processed/L6_processed.csv', 'r') as file:
    csv_reader = csv.DictReader(file, fieldnames=['author', 'review', 'date_review', 'note', 'cdiscount_response', 'cdiscount_response_date', 'cdiscount_response_content', 'polarite', 'subjectivite'])
    next(csv_reader)  # Skip the header row
    
    for row in csv_reader:
        # Check if date fields have valid values
        if not row['date_review']:
            row['date_review'] = '01/01/1900'
        if not row['cdiscount_response_date']:
            row['cdiscount_response_date'] = '01/01/1900'
        
        # Calculate the scaled note value
        note = int(row['note'])
        note_p = (note - 1) / 4
        
        entry = {
            'author': row['author'],
            'review': row['review'],
            'date_review': row['date_review'],
            'note': note,
            'note_p': note_p,
            'cdiscount_response': int(row['cdiscount_response']),
            'cdiscount_response_date': row['cdiscount_response_date'],
            'cdiscount_response_content': row['cdiscount_response_content'],
            'polarite': float(row['polarite']),
            'subjectivite': float(row['subjectivite'])
        }
        
        es.index(index=index_name, body=entry)

/tmp/ipykernel_27870/2348605399.py:52: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=mapping, ignore=400)


In [11]:
from elasticsearch import Elasticsearch
import csv

index_name = "test_relationnel"

mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "name": {"type": "text"},
            "URL": {"type": "keyword"},
            "note": {"type": "float"},
            "avis": {"type": "integer"},
            "sous-cat": {"type": "text"}
        }
    }
}

# Supprimer l'index existant
es.indices.delete(index=index_name, ignore=[400, 404])

# Créer l'index avec le nouveau mapping
es.indices.create(index=index_name, body=mapping)

# Réindexer les données à partir du fichier CSV
with open('/home/jben/Documents/CDE_Satisfaction2023/data/processed/L4_processed.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    next(csv_reader)  # Skip the header row
    
    for row in csv_reader:
        entry = {
            'name': row['﻿name'],
            'URL': row['URL'],
            'note': float(row['note']),
            'avis': int(row['avis']),
            'sous-cat': row['sous-cat']
        }
        
        es.index(index=index_name, body=entry)

/tmp/ipykernel_7254/1438436984.py:23: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])
